In [ ]:
!pip install sentencepiece
%load_ext autoreload
%autoreload 2
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append("/content/drive/MyDrive/neural_machine_translation/iter5B/code")
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")

print(device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.3 MB/s eta 0:00:00
Mounted at /content/drive
cuda


## LANGUAGE

In [ ]:
import sentencepiece as spm

class Language:
    def __init__(self, path_to_spm_model, lang):
        self.spm_model = spm.SentencePieceProcessor(model_file=path_to_spm_model)
        self.word2idx = self.build_word2idx()
        self.idx2word = self.build_idx2word()
        self.n_words = self.spm_model.get_piece_size()
        self.lang = lang
        self.bos_id = self.spm_model.bos_id()
        self.eos_id = self.spm_model.eos_id()
        self.pad_id = self.spm_model.pad_id()
        self.unk_id = self.spm_model.unk_id()

    def build_word2idx(self):
        word2idx = {}
        for i in range(self.spm_model.get_piece_size()):
            word2idx[self.spm_model.id_to_piece(i)] = i
        return word2idx

    def build_idx2word(self):
        idx2word = {}
        for i in range(self.spm_model.get_piece_size()):
            idx2word[i] = self.spm_model.id_to_piece(i)
        return idx2word

    def tokenize(self, sentence):
        return self.spm_model.Encode(sentence)

    def detokenize(self, indices):
        return self.spm_model.Decode(indices)

    def prune_sentences(self, sentences, len):
        # sentences: list of indexed sentences
        pruned_sentences = []
        for sent in sentences:
            if len(sent) < len:
                pruned_sentences.append(sent)
            else:
                pruned_sentences.append(sent[:len - 1])
        return pruned_sentences

    def normalize_sentences(self, sentences, len):
        # sentences: list of indexed sentences
        normalized_sentences = []
        for sentence in sentences:
            if len(sentence) < len:
                n_sentence = sentence + [self.pad_id] * (len - len(sentence))
            else:
                n_sentence = sentence[:len - 1] + [self.eos_id]
            normalized_sentences.append(n_sentence)
        return normalized_sentences

## DATALOADERS

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import json
from sklearn.model_selection import train_test_split
import numpy as np

def causal_mask(seq_len):
    mask = torch.triu(torch.ones(1, seq_len, seq_len), diagonal=1).type(torch.int)
    return mask == 0

class BilingualDataset(Dataset):
    def __init__(self, src_lang, tgt_lang, ds, seq_len=75):
        super().__init__()
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang
        self.ds = ds
        self.seq_len = seq_len

        self.sos_token = torch.tensor([self.src_lang.bos_id], dtype=torch.int64)
        self.eos_token = torch.tensor([self.src_lang.eos_id], dtype=torch.int64)
        self.pad_token = torch.tensor([self.src_lang.pad_id], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_tgt_pair = self.ds[idx]
        src_text = src_tgt_pair["src"]
        tgt_text = src_tgt_pair["tgt"]

        src_indices = self.src_lang.tokenize(src_text)
        tgt_indices = self.tgt_lang.tokenize(tgt_text)

        enc_num_padding_tokens = self.seq_len - len(src_indices) - 2
        dec_num_padding_tokens = self.seq_len - len(tgt_indices) - 1

        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(src_indices, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64)
            ],
            dim=0,
        )
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(tgt_indices, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
            ],
            dim=0,
        )
        label = torch.cat(
            [
                torch.tensor(tgt_indices, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64)
            ],
            dim=0,
        )
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,
            "decoder_input": decoder_input,
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(),
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)),
            "label": label,
            "src_text": src_text,
            "tgt_text": tgt_text
        }

def get_dataloader(path_to_raw_data, lang_pair, src_lang, tgt_lang, batch_size, train_val_split=0.1, seq_len=75):
    with open(path_to_raw_data) as f:
        raw_data = json.load(f)

    DATA = [] # list of dicts of the form {"src": ..., "tgt": ...}

    for lp, data in raw_data.items():
        for _, items in data.items():
            for _, pair in items.items():
                if lp == lang_pair:
                    src_sent = pair["source"]
                    tgt_sent = pair["target"]
                    DATA.append({"src": src_sent, "tgt": tgt_sent})

#     src_lens = [len(src_lang.tokenize(pair["src"])) for pair in DATA]
#     tgt_lens = [len(tgt_lang.tokenize(pair["tgt"])) for pair in DATA]

#     src_max_len = int(np.quantile(src_lens, pruning_quantile))
#     tgt_max_len = int(np.quantile(tgt_lens, pruning_quantile))

#     seq_len = max(src_max_len, tgt_max_len)
    DATA = [pair for pair in DATA if len(src_lang.tokenize(pair["src"])) < seq_len - 2 and len(tgt_lang.tokenize(pair["tgt"])) < seq_len - 2]

    TRAIN_DATA, VAL_DATA = train_test_split(DATA, test_size=train_val_split, random_state=42)

    train_ds = BilingualDataset(src_lang, tgt_lang, TRAIN_DATA, seq_len)
    val_ds = BilingualDataset(src_lang, tgt_lang, VAL_DATA, seq_len)

    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_ds, batch_size=1, shuffle=True)

    return train_dl, val_dl

## MODEL ARCHITECTURE

In [ ]:
import torch
import torch.nn as nn
import math

class InputEmbeddings(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model
        self.vocab_size = vocab_size
    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, seq_len, dropout):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

class LayerNormalization(nn.Module):
    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features))
        self.bias = nn.Parameter(torch.zeros(features))

    def forward(self, x):
        x = x.float()
        mean = x.mean(dim = -1, keepdim = True)
        std = x.std(dim = -1, keepdim = True)
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

class FeedForwardBlock(nn.Module):
    def __init__(self, d_model, d_ff, dropout):
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

class MultiHeadAttentionBlock(nn.Module):
    def __init__(self, d_model, h, dropout):
        super().__init__()
        # h: number of heads
        self.d_model = d_model
        self.h = h
        assert d_model % h == 0, "d_model must be divisible by h"
        self.d_k = d_model // h
        self.w_q = nn.Linear(d_model, d_model, bias=False)
        self.w_k = nn.Linear(d_model, d_model, bias=False)
        self.w_v = nn.Linear(d_model, d_model, bias=False)
        self.w_o = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout):
        d_k = query.shape[-1]
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            attention_scores = attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1)
        if dropout is not None:
            attention_scores = dropout(attention_scores)

        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q)
        key = self.w_k(k)
        value = self.w_v(v)

        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.d_model)
        return self.w_o(x)

class ResidualConnection(nn.Module):
    def __init__(self, features, dropout):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization(features)

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

class EncoderBlock(nn.Module):
    def __init__(self, features, self_attention_block, feed_forward_block, dropout):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

class Encoder(nn.Module):
    def __init__(self, features, layers):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderBlock(nn.Module):
    def __init__(self, features, self_attention_block, cross_attention_block, feed_forward_block, dropout):
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

class Decoder(nn.Module):
    def __init__(self, features, layers):
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

class ProjectionLayer(nn.Module):
    def __init__(self, d_model, vocab_size):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        return torch.log_softmax(self.proj(x), dim=-1)

class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output, src_mask, tgt, tgt_mask):
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def projection(self, x):
        return self.projection_layer(x)



def build_transformer(src_vocab_size, tgt_vocab_size, src_seq_len, tgt_seq_len, d_model = 512, N=6, h=8, dropout=0.1, d_ff=2048):
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

## TRANSLATION

In [ ]:
from tqdm import tqdm

def greedy_decode(model, source, source_mask, max_len, src_lang, tgt_lang, device):
    sos_id = src_lang.bos_id
    eos_id = src_lang.eos_id

    encoder_output = model.encode(source, source_mask)
    decoder_input = torch.empty(1, 1).fill_(sos_id).type_as(source).to(device)

    while True:
        if decoder_input.size(1) >= max_len:
            break
        decoder_mask = causal_mask(decoder_input.size(1)).type_as(source).to(device)
        decoder_output = model.decode(encoder_output, source_mask, decoder_input, decoder_mask)
        prob = model.projection(decoder_output[:, -1])
        _, next_word = torch.max(prob, dim=1)
        decoder_input = torch.cat(
            [decoder_input, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1
        )
        if next_word == eos_id:
            break

    return decoder_input.squeeze(0)

def beam_search_decode(model, source, source_mask, max_len, src_lang, tgt_lang, device, beam_size=3):
    sos_id = src_lang.bos_id
    eos_id = src_lang.eos_id
    encoder_output = model.encode(source, source_mask)

    # Initialize the sequences with the start-of-sequence token
    init_seq = torch.empty(1, 1).fill_(sos_id).type_as(source).to(device)
    sequences = [(init_seq, 0)]  # each sequence in the form (seq, score)

    for _ in range(max_len):
        all_candidates = []

        for seq, score in sequences:
            if seq[0, -1] == eos_id:
                all_candidates.append((seq, score))
                continue

            decoder_mask = causal_mask(seq.size(1)).type_as(source).to(device)
            decoder_output = model.decode(encoder_output, source_mask, seq, decoder_mask)
            prob = model.projection(decoder_output[:, -1])
            log_prob, indices = torch.topk(prob, beam_size)

            # Add top sequences and their scores
            for k in range(beam_size):
                next_word = indices[0][k]
                new_seq = torch.cat([seq, torch.empty(1, 1).type_as(source).fill_(next_word.item()).to(device)], dim=1)
                new_score = score + log_prob[0][k].item()
                all_candidates.append((new_seq, new_score))

        # Order all sequences by their scores and keep the top-k sequences
        ordered = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)
        sequences = ordered[:beam_size]

    return sequences[0][0].squeeze(0)  # Return the best sequence

def evaluate(model, src_sents, src_lang, tgt_lang, max_len, device, print_interval=100, beam_size=3):
    translated_sents = []
    model.eval()
    sos_token = torch.tensor([src_lang.bos_id], dtype=torch.int64)
    eos_token = torch.tensor([src_lang.eos_id], dtype=torch.int64)
    pad_token = torch.tensor([src_lang.pad_id], dtype=torch.int64)

    src_sents_i = [src_lang.tokenize(sent) for sent in src_sents]
    len_i = [len(sent_i) for sent_i in src_sents_i]

    pruned_sents_i = []
    for sent_i in src_sents_i:
        if len(sent_i) > max_len - 2:
            sent_i = sent_i[:max_len - 2]
        pruned_sents_i.append(sent_i)

    print(f"Translating {len(src_sents)} sentences...")

    with torch.no_grad():
        for i, src_sent_i in enumerate(tqdm(pruned_sents_i)):  # Wrap pruned_sents_i with tqdm
            num_padding = max_len - len(src_sent_i) - 2
            encoder_input = torch.cat(
                [
                    sos_token,
                    torch.tensor(src_sent_i, dtype=torch.int64),
                    eos_token,
                    torch.tensor([pad_token] * num_padding, dtype=torch.int64)
                ],
                dim=0,
            ).to(device)
            encoder_mask = (encoder_input != pad_token.to(device)).unsqueeze(0).unsqueeze(0).int().to(device)
            out = beam_search_decode(model, encoder_input, encoder_mask, max_len, src_lang, tgt_lang, device, beam_size)
            decoded_text = tgt_lang.detokenize(out.tolist())
            translated_sents.append(decoded_text)

    return translated_sents

In [ ]:
import json
def test_data(path_to_file, lang_pair):
    ids = []
    sents = []

    with open(path_to_file) as f:
        raw_data = json.load(f)

    for lp, data in raw_data.items():
        for _, items in data.items():
            for id, struct in items.items():
                if lp == lang_pair:
                    ids.append(int(id))
                    sents.append(struct["source"])

    return sents, ids

In [ ]:
tgt_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/en_spm.model", lang="en")

In [ ]:
tgt_lang.idx2word[457]

'▁cons'

In [ ]:
seq_len = 75

In [ ]:
path_to_file = "/content/drive/MyDrive/neural_machine_translation/val_data2.json"

# HINDI
src_sents, src_ids = test_data(path_to_file, "English-Hindi")
src_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/hi_spm.model", lang="hi")

In [ ]:
D_MODEL = 256
NUM_LAYERS = 6
NUM_HEADS = 8
D_FF = 1024
model = build_transformer(src_lang.n_words, tgt_lang.n_words, seq_len, seq_len, D_MODEL, NUM_LAYERS, NUM_HEADS, 0.1, D_FF).to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter7/hi_to_en_seq2seq_iter7.pt"))

<All keys matched successfully>

In [ ]:
translated_sents = evaluate(model, src_sents, src_lang, tgt_lang, seq_len, device)

Translating 11542 sentences...


100%|██████████| 11542/11542 [2:02:24<00:00,  1.57it/s]


In [ ]:
import pandas as pd
df = pd.DataFrame({'ID': src_ids, 'Translation': translated_sents})

csv_path = "/content/drive/MyDrive/neural_machine_translation/iter7/hindi.csv"
df.to_csv(csv_path, index=False)

In [ ]:
# BENGALI
src_sents, src_ids = test_data(path_to_file, "English-Bengali")
src_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/be_spm.model", lang="be")

In [ ]:
D_MODEL = 256
NUM_LAYERS = 6
NUM_HEADS = 8
D_FF = 1024
model = build_transformer(src_lang.n_words, tgt_lang.n_words, seq_len, seq_len, D_MODEL, NUM_LAYERS, NUM_HEADS, 0.1, D_FF).to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter7/be_to_en_seq2seq_iter7.pt"))

<All keys matched successfully>

In [ ]:
translated_sents = evaluate(model, src_sents, src_lang, tgt_lang, seq_len, device)

Translating 9835 sentences...


100%|██████████| 9835/9835 [1:43:48<00:00,  1.58it/s]


In [ ]:
df = pd.DataFrame({'ID': src_ids, 'Translation': translated_sents})

csv_path = "/content/drive/MyDrive/neural_machine_translation/iter7/bengali.csv"
df.to_csv(csv_path, index=False)

In [ ]:
# GUJARATI
src_sents, src_ids = test_data(path_to_file, "English-Gujarati")
src_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/gu_spm.model", lang="gu")

In [ ]:
D_MODEL = 256
NUM_LAYERS = 6
NUM_HEADS = 8
D_FF = 1024
model = build_transformer(src_lang.n_words, tgt_lang.n_words, seq_len, seq_len, D_MODEL, NUM_LAYERS, NUM_HEADS, 0.1, D_FF).to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter7/gu_to_en_seq2seq_iter7.pt"))

<All keys matched successfully>

In [ ]:
translated_sents = evaluate(model, src_sents, src_lang, tgt_lang, seq_len, device)

Translating 6783 sentences...


100%|██████████| 6783/6783 [1:15:49<00:00,  1.49it/s]


In [ ]:
df = pd.DataFrame({'ID': src_ids, 'Translation': translated_sents})

csv_path = "/content/drive/MyDrive/neural_machine_translation/iter7/gujarati.csv"
df.to_csv(csv_path, index=False)

In [ ]:
# KANNADA
src_sents, src_ids = test_data(path_to_file, "English-Kannada")
src_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/ka_spm.model", lang="ka")

In [ ]:
D_MODEL = 256
NUM_LAYERS = 6
NUM_HEADS = 8
D_FF = 1024
model = build_transformer(src_lang.n_words, tgt_lang.n_words, seq_len, seq_len, D_MODEL, NUM_LAYERS, NUM_HEADS, 0.1, D_FF).to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter7/ka_to_en_seq2seq_iter7.pt"))

<All keys matched successfully>

In [ ]:
translated_sents = evaluate(model, src_sents, src_lang, tgt_lang, seq_len, device)

In [ ]:
df = pd.DataFrame({'ID': src_ids, 'Translation': translated_sents})

csv_path = "/content/drive/MyDrive/neural_machine_translation/iter7/kannada.csv"
df.to_csv(csv_path, index=False)

In [ ]:
# MALAYALAM
src_sents, src_ids = test_data(path_to_file, "English-Malayalam")
src_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/ma_spm.model", lang="ma")

In [ ]:
D_MODEL = 256
NUM_LAYERS = 6
NUM_HEADS = 8
D_FF = 1024
model = build_transformer(src_lang.n_words, tgt_lang.n_words, seq_len, seq_len, D_MODEL, NUM_LAYERS, NUM_HEADS, 0.1, D_FF).to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter7/ma_to_en_seq2seq_iter7.pt"))

In [ ]:
translated_sents = evaluate(model, src_sents, src_lang, tgt_lang, seq_len, device)

In [ ]:
df = pd.DataFrame({'ID': src_ids, 'Translation': translated_sents})

csv_path = "/content/drive/MyDrive/neural_machine_translation/iter7/malayalam.csv"
df.to_csv(csv_path, index=False)

In [ ]:
# TAMIL
src_sents, src_ids = test_data(path_to_file, "English-Tamil")
src_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/ta_spm.model", lang="ta")

In [ ]:
D_MODEL = 256
NUM_LAYERS = 6
NUM_HEADS = 8
D_FF = 1024
model = build_transformer(src_lang.n_words, tgt_lang.n_words, seq_len, seq_len, D_MODEL, NUM_LAYERS, NUM_HEADS, 0.1, D_FF).to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter7/ta_to_en_seq2seq_iter7.pt"))

In [ ]:
translated_sents = evaluate(model, src_sents, src_lang, tgt_lang, seq_len, device)

In [ ]:
df = pd.DataFrame({'ID': src_ids, 'Translation': translated_sents})

csv_path = "/content/drive/MyDrive/neural_machine_translation/iter7/tamil.csv"
df.to_csv(csv_path, index=False)

In [ ]:
# TELGU
src_sents, src_ids = test_data(path_to_file, "English-Telgu")
src_lang = Language("/content/drive/MyDrive/neural_machine_translation/iter5B/spm/te_spm.model", lang="te")

In [ ]:
D_MODEL = 256
NUM_LAYERS = 6
NUM_HEADS = 8
D_FF = 1024
model = build_transformer(src_lang.n_words, tgt_lang.n_words, seq_len, seq_len, D_MODEL, NUM_LAYERS, NUM_HEADS, 0.1, D_FF).to(device)

model.load_state_dict(torch.load("/content/drive/MyDrive/neural_machine_translation/iter7/te_to_en_seq2seq_iter7.pt"))

In [ ]:
translated_sents = evaluate(model, src_sents, src_lang, tgt_lang, seq_len, device)

In [ ]:
df = pd.DataFrame({'ID': src_ids, 'Translation': translated_sents})

csv_path = "/content/drive/MyDrive/neural_machine_translation/iter7/telgu.csv"
df.to_csv(csv_path, index=False)